Name: Anne Sai Venkata Naga Saketh <br>
USC ID: 3725520208 <br>
USC Email: annes@usc.edu <br>
<b> NLP HW2 </b>

## Import Statements

In [1]:
# Importing necessary libraries
import pandas as pd  # For data manipulation and analysis
import numpy as np   # For numerical operations
import re            # Regular expressions for text processing
from bs4 import BeautifulSoup  # For HTML parsing
from sklearn.model_selection import train_test_split  # For splitting data into training and testing sets
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

import nltk          # Natural Language Toolkit for text processing
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')  # Download WordNet data
nltk.download('stopwords')   # Download StopWords data

import warnings      # To handle warnings
warnings.filterwarnings("ignore")  # Ignore warnings for the remainder of the code
warnings.filterwarnings("default")  # Set warnings back to default behavior

import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from gensim.models import KeyedVectors
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec

from nltk.tokenize import word_tokenize
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical
from sklearn.linear_model import Perceptron
from sklearn.svm import LinearSVC

import torch.nn as nn
import torch.optim as optim

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sakethanne/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sakethanne/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Read Data

In [2]:
# Reading the data from the tsv (Amazon Kitchen dataset) file as a Pandas frame
full_data = pd.read_csv("./amazon_reviews_us_Office_Products_v1_00.tsv", delimiter='\t', encoding='utf-8', error_bad_lines=False)

/var/folders/xx/d1tzxzhj3fzbmswz4z7m_40w0000gn/T/ipykernel_12747/1150709939.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  full_data = pd.read_csv("./amazon_reviews_us_Office_Products_v1_00.tsv", delimiter='\t', encoding='utf-8', error_bad_lines=False)
Skipping line 20773: expected 15 fields, saw 22
Skipping line 39834: expected 15 fields, saw 22
Skipping line 52957: expected 15 fields, saw 22
Skipping line 54540: expected 15 fields, saw 22

Skipping line 80276: expected 15 fields, saw 22
Skipping line 96168: expected 15 fields, saw 22
Skipping line 96866: expected 15 fields, saw 22
Skipping line 98175: expected 15 fields, saw 22
Skipping line 112539: expected 15 fields, saw 22
Skipping line 119377: expected 15 fields, saw 22
Skipping line 120065: expected 15 fields, saw 22
Skipping line 124703: expected 15 fields, saw 22

Skipping line 134024: expected 15 fields, saw 22
Skipping line 15

## Extract only Review and Ratings

In [3]:
# Printing the data frame that contains the entire dataset from the tsv file
print(full_data)

# Keep only the Reviews and Ratings fields from the full data
df = full_data[['review_body', 'star_rating']]

# Converting 'star_rating' to numeric values
df['star_rating'] = pd.to_numeric(full_data['star_rating'], errors='coerce')

        marketplace  customer_id       review_id  product_id  product_parent  \
0                US     43081963  R18RVCKGH1SSI9  B001BM2MAC       307809868   
1                US     10951564  R3L4L6LW1PUOFY  B00DZYEXPQ        75004341   
2                US     21143145  R2J8AWXWTDX2TF  B00RTMUHDW       529689027   
3                US     52782374  R1PR37BR7G3M6A  B00D7H8XB6       868449945   
4                US     24045652  R3BDDDZMZBZDPU  B001XCWP34        33521401   
...             ...          ...             ...         ...             ...   
2640249          US     53005790   RLI7EI10S7SN0  B00000DM9M       223408988   
2640250          US     52188548  R1F3SRK9MHE6A3  B00000DM9M       223408988   
2640251          US     52090046  R23V0C4NRJL8EM  0807865001       307284585   
2640252          US     52503173  R13ZAE1ATEUC1T  1572313188       870359649   
2640253          US     52585611   RE8J5O2GY04NN  1572313188       870359649   

                                       

/var/folders/xx/d1tzxzhj3fzbmswz4z7m_40w0000gn/T/ipykernel_12747/79198575.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['star_rating'] = pd.to_numeric(full_data['star_rating'], errors='coerce')


## Prepare a Balanced Data Set and prepare a Testing and Training Set

In [4]:
# Check unique values in 'star_rating' column
unique_ratings = df['star_rating'].unique()
# print("Unique ratings:", unique_ratings)

# Convert 'star_rating' column to integer, handling errors
df['star_rating'] = pd.to_numeric(df['star_rating'], errors='coerce')

# Drop rows with NaN values in 'star_rating' column
df = df.dropna(subset=['star_rating'])

# Convert 'star_rating' column to integer
df['star_rating'] = df['star_rating'].astype(int)

# Define a custom dataset class
class AmazonReviewsDataset(Dataset):
    def __init__(self, reviews, ratings):
        self.reviews = reviews
        self.ratings = ratings

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
        review = self.reviews[idx]
        rating = self.ratings[idx]
        return review, rating

# Build balanced dataset
ratings = df['star_rating'].unique()
balanced_data = pd.DataFrame(columns=df.columns)
for rating in ratings:
    subset = df[df['star_rating'] == rating]
    if len(subset) >= 50000:
        subset = subset.sample(n=50000, random_state=42)
    balanced_data = pd.concat([balanced_data, subset])

# Create ternary labels
balanced_data['sentiment'] = np.where(balanced_data['star_rating'] > 3, 1, 
                                      np.where(balanced_data['star_rating'] < 3, 2, 3))

print("Checking if the Dataset has been balanced out:\n")
print("Star_rating  Count")
print(balanced_data['star_rating'].value_counts())

print("Checking if the Sentiments has been balanced out:\n")
print("Sentiment  Count")
print(balanced_data['sentiment'].value_counts())

# Perform train-test split
train_data, test_data = train_test_split(balanced_data, test_size=0.2, random_state=42)

# Define train and test datasets
train_dataset = AmazonReviewsDataset(train_data['review_body'].values, train_data['sentiment'].values)
test_dataset = AmazonReviewsDataset(test_data['review_body'].values, test_data['sentiment'].values)

# Define DataLoader
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Print sizes of train and test datasets
print("\nPrinting the Test and the Training set data sizes")
print("Train dataset size:", len(train_dataset))
print("Test dataset size:", len(test_dataset))

/var/folders/xx/d1tzxzhj3fzbmswz4z7m_40w0000gn/T/ipykernel_12747/2411086817.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['star_rating'] = pd.to_numeric(df['star_rating'], errors='coerce')


Checking if the Dataset has been balanced out:

Star_rating  Count
5    50000
1    50000
4    50000
2    50000
3    50000
Name: star_rating, dtype: int64
Checking if the Sentiments has been balanced out:

Sentiment  Count
1    100000
2    100000
3     50000
Name: sentiment, dtype: int64

Printing the Test and the Training set data sizes
Train dataset size: 200000
Test dataset size: 50000


## Split the Training and Testing Data Set

In [5]:
# Splitting the dataset into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(balanced_data['review_body'],
                                                    balanced_data['sentiment'],
                                                    test_size=0.2,
                                                    random_state=42)

## Clean the Data

In [6]:
# Define a contraction map
CONTRACTION_MAP = {
    "won't": "will not",
    "can't": "cannot",
    "i'm": "i am",
    "you're": "you are",
    "he's": "he is",
    "she's": "she is",
    "it's": "it is",
    "that's": "that is",
    "we're": "we are",
    "they're": "they are",
    "isn't": "is not",
    "aren't": "are not",
    "haven't": "have not",
    "hasn't": "has not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "wasn't": "was not",
    "weren't": "were not",
    "haven't": "have not",
    "hasn't": "has not",
    "won't've": "will not have",
    "can't've": "cannot have",
    "i'll": "i will",
    "you'll": "you will",
    "he'll": "he will",
    "she'll": "she will",
    "it'll": "it will",
    "that'll": "that will",
    "we'll": "we will",
    "they'll": "they will",
    "i'd": "i would",
    "you'd": "you would",
    "he'd": "he would",
    "she'd": "she would",
    "it'd": "it would",
    "that'd": "that would",
    "we'd": "we would",
    "they'd": "they would",
    "i've": "i have",
    "you've": "you have",
    "we've": "we have",
    "they've": "they have",
    "shouldn't": "should not",
    "couldn't": "could not",
    "wouldn't": "would not",
    "mightn't": "might not",
    "mustn't": "must not",
    "shan't": "shall not",
    "oughtn't": "ought not",
    "who's": "who is",
    "what's": "what is",
    "where's": "where is",
    "when's": "when is",
    "why's": "why is",
    "how's": "how is",
    "it's": "it is",
    "let's": "let us"
}

# Function to expand contractions
def expand_contractions(text):
    for contraction, expansion in CONTRACTION_MAP.items():
        text = re.sub(contraction, expansion, text)
    return text

# Preprocess the reviews
def preprocess_reviews(reviews):
    # Convert to lowercase and handle NaN values
    reviews = reviews.apply(lambda x: str(x).lower() if pd.notna(x) else '')
    
    # Remove HTML and URLs
    reviews = reviews.apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())
    reviews = reviews.apply(lambda x: re.sub(r'http\S+', '', x))

    # Remove non-alphabetical characters (excluding single quote)
    reviews = reviews.apply(lambda x: re.sub(r'[^a-zA-Z\s\']', '', x))

    # Remove extra spaces
    reviews = reviews.apply(lambda x: re.sub(' +', ' ', x))

    # Perform contractions
    reviews = reviews.apply(expand_contractions)

    # Return the processed text of the review
    return reviews

# Preprocess the training set
X_train_preprocessed = preprocess_reviews(X_train)

# Print average length of reviews before and after cleaning
avg_length_before = X_train.apply(lambda x: len(str(x))).mean()
avg_length_after = X_train_preprocessed.apply(len).mean()
print("===================Printing the Average lenght of Reviews Before and After Cleaning====================")
print(f"\nAverage Length of Reviews (Before Cleaning): {int(avg_length_before)} characters")
print(f"Average Length of Reviews (After Cleaning): {int(avg_length_after)} characters")

/var/folders/xx/d1tzxzhj3fzbmswz4z7m_40w0000gn/T/ipykernel_12747/3895173410.py:75: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  reviews = reviews.apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())
/var/folders/xx/d1tzxzhj3fzbmswz4z7m_40w0000gn/T/ipykernel_12747/3895173410.py:75: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  reviews = reviews.apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())


===================Printing the Average lenght of Reviews Before and After Cleaning====================

Average Length of Reviews (Before Cleaning): 343 characters
Average Length of Reviews (After Cleaning): 326 characters


## Pre-Process the Data

In [7]:
# Initialize NLTK's stopwords and WordNet lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Function to remove stop words and perform lemmatization
def preprocess_nltk(review):
    if pd.notna(review):
        words = nltk.word_tokenize(str(review).lower())  # Convert to lowercase
        words = [lemmatizer.lemmatize(word) for word in words if word.isalpha() and word not in stop_words]
        return ' '.join(words)
    else:
        return ''

# Preprocess the training set using NLTK
X_train_nltk_preprocessed = X_train_preprocessed.apply(preprocess_nltk)

# Print three sample reviews before and after NLTK preprocessing
sample_reviews_indices = X_train_preprocessed.sample(3).index

print("============ Printing Sample Reviews Before and After Pre-processing =============")
for index in sample_reviews_indices:
    print(f"\nSample Review {index} Before Pre-processing:")
    print(X_train_preprocessed.loc[index])

    print(f"\nSample Review {index} After NLTK Pre-processing:")
    print(X_train_nltk_preprocessed.loc[index])

# Print average length of reviews before and after NLTK processing
avg_length_before_nltk = X_train_preprocessed.apply(len).mean()
avg_length_after_nltk = X_train_nltk_preprocessed.apply(len).mean()
print("\n=================Printing the Average lenght of Reviews Before and After Pre-processing==================")
print(f"\nAverage Length of Reviews (Before NLTK Processing): {int(avg_length_before_nltk)} characters")
print(f"Average Length of Reviews (After NLTK Processing): {int(avg_length_after_nltk)} characters")

============ Printing Sample Reviews Before and After Pre-processing =============

Sample Review 149348 Before Pre-processing:
very good fit replacement battery

Sample Review 149348 After NLTK Pre-processing:
good fit replacement battery

Sample Review 931199 Before Pre-processing:
i have not had it that long and it will not hold a charge actually will not charge even got a new battery and that is not the problem i will probably just have to trash it and get something else when it worked it was fine

Sample Review 931199 After NLTK Pre-processing:
long hold charge actually charge even got new battery problem probably trash get something else worked fine

Sample Review 652920 Before Pre-processing:
i am very pleased with the product and the price

Sample Review 652920 After NLTK Pre-processing:
pleased product price

=================Printing the Average lenght of Reviews Before and After Pre-processing==================

Average Length of Reviews (Before NLTK Processing): 326 charact

 ##  Word Embedding

In [8]:
# !pip install gensim

## Similarity Score using the Word2Vec Pretrained model

In [9]:
# Joining features and targets for training dataset
train_data = pd.concat([X_train_nltk_preprocessed, y_train], axis=1)
train_data_filtered = train_data[train_data['sentiment'] != 3]
X_train_binary = train_data_filtered['review_body']
y_train_binary = train_data_filtered['sentiment']

# Joining features and targets for testing dataset
test_data = pd.concat([X_test, y_test], axis=1)
test_data_filtered = test_data[test_data['sentiment'] != 3]
X_test_binary = test_data_filtered['review_body']
y_test_binary = test_data_filtered['sentiment']

In [10]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

In [11]:
# Function to extract Word2Vec features for a given sentence
def extract_word2vec_features(sentence, model, vector_size):
    word_vectors = []
    for word in sentence:
        if word in model.key_to_index:
            word_vectors.append(model.get_vector(word))
    if len(word_vectors) == 0:
        return np.zeros(vector_size)  # Return zero vector if no word vectors found
    else:
        return np.mean(word_vectors, axis=0)  # Return average word vector

# Examples to check semantic similarities
example1 = wv.most_similar(positive=['king', 'woman'], negative=['man'], topn=1)
print("King - Man + Woman =", example1)

example2 = wv.most_similar(positive=['excellent', 'outstanding'], topn=1)
print("Excellent ~ Outstanding =", example2)

King - Man + Woman = [('queen', 0.7118192911148071)]
Excellent ~ Outstanding = [('oustanding', 0.750198483467102)]


## Similarity Score using the Word2Vec model with the custom model

In [12]:
X_train_nltk_preprocessed_new = []

for e, k in tqdm(enumerate(X_train_nltk_preprocessed.to_list())):
    try:
        X_train_nltk_preprocessed_new.append(word_tokenize(k))
    except:
        pass    
def get_doc_embedding(doc):
    words = doc.lower().split()
    return wv.get_mean_vector(words)

200000it [00:21, 9387.12it/s] 


In [13]:
from gensim.models import Word2Vec
# Train the Word2Vec model
model_own = Word2Vec(X_train_nltk_preprocessed_new, vector_size=300, window=11, min_count=10)

In [14]:
# Function to extract Word2Vec features for a given sentence using the trained model
def extract_word2vec_features_own(sentence, model, vector_size):
    word_vectors = []
    for word in sentence:
        if word in model.wv.key_to_index:
            word_vectors.append(model.wv.get_vector(word))
    if len(word_vectors) == 0:
        return np.zeros(vector_size)  # Return zero vector if no word vectors found
    else:
        return np.mean(word_vectors, axis=0)  # Return average word vector

# Examples to check semantic similarities using your own model
if 'good' in model_own.wv.key_to_index:
    example1_own = model_own.wv.most_similar(positive=['good'], negative=['bad'], topn=1)
    print("King - Man + Woman (Own Model) =", example1_own)
else:
    print("'good' is not present in the vocabulary.")

if 'excellent' in model_own.wv.key_to_index and 'outstanding' in model_own.wv.key_to_index:
    example2_own = model_own.wv.most_similar(positive=['excellent', 'outstanding'], topn=1)
    print("Excellent ~ Outstanding (Own Model) =", example2_own)
else:
    print("'excellent' and/or 'outstanding' are not present in the vocabulary.")

# Compare semantic similarities between pretrained and own models
print("\nSemantic similarity (Pretrained Model):", example1)
print("Semantic similarity (Pretrained Model):", example2)

King - Man + Woman (Own Model) = [('excellent', 0.47700005769729614)]
Excellent ~ Outstanding (Own Model) = [('superb', 0.8427599668502808)]

Semantic similarity (Pretrained Model): [('queen', 0.7118192911148071)]
Semantic similarity (Pretrained Model): [('oustanding', 0.750198483467102)]


## What do you conclude from comparing vectors generated by yourself and the pretrained model? Which of the Word2Vec models seems to encode semantic similarities between words better?

Comparing vectors generated by pre-trained Word2Vec models and those trained on specific datasets reveals nuanced trade-offs. Pre-trained models, leveraging vast corpora, excel in capturing broad semantic similarities but may lack fine-grained domain specificity. Conversely, self-generated vectors, tailored to specific contexts, offer potential for domain-specific insights but require representative data and entail computational costs. Evaluating both models on relevant tasks like word similarity or downstream applications is crucial to determining which better encodes semantic relationships for specific use cases.

## Extract the TF-IDF Features from Dataset and train Perceptron and SVM

In [15]:
# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=2000000)

# Fit and transform the training set
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_binary)

# Transform the test set
X_test_tfidf = tfidf_vectorizer.transform(X_test_binary.apply(preprocess_nltk))

# Print the shape of the TF-IDF matrices
print(f"\nShape of X_train_tfidf: {X_train_tfidf.shape}")
print(f"Shape of X_test_tfidf: {X_test_tfidf.shape}")


Shape of X_train_tfidf: (160200, 112187)
Shape of X_test_tfidf: (39800, 112187)


In [16]:
perceptron = Perceptron(max_iter=1000)

perceptron.fit(X_train_tfidf, y_train_binary)
y_pred = perceptron.predict(X_test_tfidf)
accuracy_perceptron_tf_idf = accuracy_score(y_test_binary, y_pred)
print(f"Accuracy of the Single Layer Perceptron using TF-IDF Features: {accuracy_perceptron_tf_idf}")

Accuracy of the Single Layer Perceptron using TF-IDF Features: 0.8129396984924623


In [17]:
svm = LinearSVC()

svm.fit(X_train_tfidf, y_train_binary)
y_pred = svm.predict(X_test_tfidf)
accuracy_svm_tf_idf = accuracy_score(y_test_binary, y_pred)
print(f"Accuracy of the SVM using TF-IDF Features: {accuracy_svm_tf_idf}")

Accuracy of the SVM using TF-IDF Features: 0.8603015075376884


## Extract the Word2Vec Pre-trained Features from Dataset and train Perceptron and SVM

In [18]:
def get_doc_embedding(doc):
    doc_str = ' '.join(doc)  # Join tokens within the document list to form a single string
    words = doc_str.lower().split()  # Split the string into words
    if not words:
        return np.zeros(300)  # Return a zero vector if no words are present
    else:
        return wv.get_mean_vector(words)  # Compute the mean vector using Word2Vec model

X_train_w2v_binary = []
X_test_w2v_binary = []

# Extract Word2Vec features for training data
for e in tqdm(X_train_binary):
    X_train_w2v_binary.append(get_doc_embedding(e))

# Extract Word2Vec features for testing data
for e in tqdm(X_test_binary):
    X_test_w2v_binary.append(get_doc_embedding(e))
    
X_train_w2v_binary = np.array(X_train_w2v_binary)
X_test_w2v_binary = np.array(X_test_w2v_binary)

X_train_w2v_binary.shape, X_test_w2v_binary.shape

100%|███████████████████████████████████| 39800/39800 [00:34<00:00, 1150.79it/s]


((160200, 300), (39800, 300))

In [19]:
# y_train_one_hot = to_categorical(y_train_binary)
# y_test_one_hot = to_categorical(y_test_binary)

In [20]:
perceptron = Perceptron(max_iter=1000)

perceptron.fit(X_train_w2v_binary, y_train_binary)
y_pred = perceptron.predict(X_test_w2v_binary)
accuracy_perceptron_w2v_pre = accuracy_score(y_test_binary, y_pred)
print(f"Accuracy of the Single Layer Perceptron using Word2Vec Pretrained Features: {accuracy_perceptron_w2v_pre}")

Accuracy of the Single Layer Perceptron using Word2Vec Pretrained Features: 0.4999497487437186


In [21]:
svm = LinearSVC()

svm.fit(X_train_w2v_binary, y_train_binary)
y_pred = svm.predict(X_test_w2v_binary)
accuracy_svm_w2v_pre = accuracy_score(y_test_binary, y_pred)
print(f"Accuracy of the SVM using Word2Vec Pretrained Features: {accuracy_svm_w2v_pre}")

Accuracy of the SVM using Word2Vec Pretrained Features: 0.6078391959798995


## Extract the Word2Vec Custom Features from Dataset and train Perceptron and SVM

In [22]:
def get_doc_embedding(doc):
    doc_str = ' '.join(doc)  # Join tokens within the document list to form a single string
    words = doc_str.lower().split()  # Split the string into words
    if not words:
        return np.zeros(300)  # Return a zero vector if no words are present
    else:
        return model_own.wv.get_mean_vector(words)  # Compute the mean vector using Word2Vec model

X_train_w2v_own_binary = []
X_test_w2v_own_binary = []

# Extract Word2Vec features for training data
for e in tqdm(X_train_binary):
    X_train_w2v_own_binary.append(get_doc_embedding(e))

# Extract Word2Vec features for testing data
for e in tqdm(X_test_binary):
    X_test_w2v_own_binary.append(get_doc_embedding(e))
    
X_train_w2v_own_binary = np.array(X_train_w2v_own_binary)
X_test_w2v_own_binary = np.array(X_test_w2v_own_binary)

X_train_w2v_own_binary.shape, X_test_w2v_own_binary.shape

100%|███████████████████████████████████| 39800/39800 [00:30<00:00, 1294.77it/s]


((160200, 300), (39800, 300))

In [23]:
perceptron = Perceptron(max_iter=1000)

perceptron.fit(X_train_w2v_own_binary, y_train_binary)
y_pred = perceptron.predict(X_test_w2v_own_binary)
accuracy_perceptron_w2v_own = accuracy_score(y_test_binary, y_pred)
print(f"Accuracy of the Single Layer Perceptron using Word2Vec Custom Features: {accuracy_perceptron_w2v_own}")

Accuracy of the Single Layer Perceptron using Word2Vec Custom Features: 0.5013065326633166


In [24]:
svm = LinearSVC()

svm.fit(X_train_w2v_own_binary, y_train_binary)
y_pred = svm.predict(X_test_w2v_own_binary)
accuracy_svm_w2v_own = accuracy_score(y_test_binary, y_pred)
print(f"Accuracy of the SVM using Word2Vec Custom Features: {accuracy_svm_w2v_own}")

Accuracy of the SVM using Word2Vec Custom Features: 0.6078140703517588


## Multi Layer Perceptron with Binary Classification

In [27]:
# Convert labels to NumPy arrays
y_train_np = np.array(y_train_binary)
y_test_np = np.array(y_test_binary)

# Convert labels to binary format (0 for class 1, 1 for class 2)
y_train_binary = np.where(y_train_np == 1, 0, 1)
y_test_binary = np.where(y_test_np == 1, 0, 1)

# Define the MLP Model for binary classification
class BinaryMLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(BinaryMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return self.softmax(x)

# Training Loop
def train(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss}")

# Evaluation
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy: {correct/total}")

## Extract the Word2Vec Custom Features and train a Multi Layer Perceptron with Binary Classification

In [29]:
# Convert data to PyTorch tensors
train_dataset_own_binary = TensorDataset(torch.tensor(X_train_w2v_own_binary, dtype=torch.float32), torch.tensor(y_train_binary, dtype=torch.long))
test_dataset_own_binary = TensorDataset(torch.tensor(X_test_w2v_own_binary, dtype=torch.float32), torch.tensor(y_test_binary, dtype=torch.long))

# Create DataLoader
train_loader_own_binary = DataLoader(train_dataset_own_binary, batch_size=32, shuffle=True)
test_loader_own_binary = DataLoader(test_dataset_own_binary, batch_size=32)

# Initialize the model
input_size = X_train_w2v_own_binary.shape[1]  # Size of input features
hidden_size1 = 50
hidden_size2 = 10
output_size = 2  # Binary classification (classes 1 and 2)
binary_model_own_binary = BinaryMLP(input_size, hidden_size1, hidden_size2, output_size)

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(binary_model_own_binary.parameters(), lr=0.001)

# Train the binary classification model
num_epochs = 50
train(binary_model_own_binary, train_loader_own_binary, criterion, optimizer, num_epochs)

# Evaluate the binary classification model
evaluate(binary_model_own_binary, test_loader_own_binary)

Epoch 1/50, Loss: 3329.3513650894165
Epoch 2/50, Loss: 3256.3875443935394
Epoch 3/50, Loss: 3229.9809458851814
Epoch 4/50, Loss: 3217.924126148224
Epoch 5/50, Loss: 3209.0161807239056
Epoch 6/50, Loss: 3205.7984357476234
Epoch 7/50, Loss: 3199.1441574692726
Epoch 8/50, Loss: 3196.9826451539993
Epoch 9/50, Loss: 3194.0523723959923
Epoch 10/50, Loss: 3188.603025317192
Epoch 11/50, Loss: 3181.285680025816
Epoch 12/50, Loss: 3177.8896557092667
Epoch 13/50, Loss: 3175.050920009613
Epoch 14/50, Loss: 3171.7565844357014
Epoch 15/50, Loss: 3168.9987232983112
Epoch 16/50, Loss: 3168.3483400046825
Epoch 17/50, Loss: 3164.3925160467625
Epoch 18/50, Loss: 3164.463406264782
Epoch 19/50, Loss: 3163.1284128427505
Epoch 20/50, Loss: 3161.315751582384
Epoch 21/50, Loss: 3158.9698126912117
Epoch 22/50, Loss: 3154.1887883245945
Epoch 23/50, Loss: 3152.3663188517094
Epoch 24/50, Loss: 3149.1161662340164
Epoch 25/50, Loss: 3148.362521737814
Epoch 26/50, Loss: 3146.098133325577
Epoch 27/50, Loss: 3145.70733

## Extract the Word2Vec Pretrained Features and train a Multi Layer Perceptron with Binary Classification

In [30]:
# Convert data to PyTorch tensors
train_dataset_pre_binary = TensorDataset(torch.tensor(X_train_w2v_binary, dtype=torch.float32), torch.tensor(y_train_binary, dtype=torch.long))
test_dataset_pre_binary = TensorDataset(torch.tensor(X_test_w2v_binary, dtype=torch.float32), torch.tensor(y_test_binary, dtype=torch.long))

# Create DataLoader
train_loader_pre_binary = DataLoader(train_dataset_pre_binary, batch_size=32, shuffle=True)
test_loader_pre_binary = DataLoader(test_dataset_pre_binary, batch_size=32)

# Initialize the model
input_size = X_train_w2v_binary.shape[1]  # Size of input features
hidden_size1 = 50
hidden_size2 = 10
output_size = 2  # Binary classification (classes 1 and 2)
binary_model_pre_binary = BinaryMLP(input_size, hidden_size1, hidden_size2, output_size)

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(binary_model_pre_binary.parameters(), lr=0.001)

# Train the binary classification model
num_epochs = 50
train(binary_model_pre_binary, train_loader_pre_binary, criterion, optimizer, num_epochs)

# Evaluate the binary classification model
evaluate(binary_model_pre_binary, test_loader_pre_binary)

Epoch 1/50, Loss: 3337.4741054177284
Epoch 2/50, Loss: 3258.777542024851
Epoch 3/50, Loss: 3229.996010273695
Epoch 4/50, Loss: 3217.8045231997967
Epoch 5/50, Loss: 3210.592474937439
Epoch 6/50, Loss: 3203.112156122923
Epoch 7/50, Loss: 3198.713766813278
Epoch 8/50, Loss: 3194.7120011150837
Epoch 9/50, Loss: 3190.2736302912235
Epoch 10/50, Loss: 3186.84253308177
Epoch 11/50, Loss: 3183.506317228079
Epoch 12/50, Loss: 3179.2488149106503
Epoch 13/50, Loss: 3176.3761309981346
Epoch 14/50, Loss: 3173.972962588072
Epoch 15/50, Loss: 3171.9340659677982
Epoch 16/50, Loss: 3169.777444243431
Epoch 17/50, Loss: 3168.372892022133
Epoch 18/50, Loss: 3163.1078752577305
Epoch 19/50, Loss: 3162.1231328547
Epoch 20/50, Loss: 3159.290839523077
Epoch 21/50, Loss: 3158.057747721672
Epoch 22/50, Loss: 3155.1658323407173
Epoch 23/50, Loss: 3152.201946377754
Epoch 24/50, Loss: 3150.5851868391037
Epoch 25/50, Loss: 3150.4111152887344
Epoch 26/50, Loss: 3146.548693239689
Epoch 27/50, Loss: 3144.7266596853733
E

## Multi Layer Perceptron with Ternary Classification

In [31]:
def get_doc_embedding(doc):
    doc_str = ' '.join(doc)  # Join tokens within the document list to form a single string
    words = doc_str.lower().split()  # Split the string into words
    if not words:
        return np.zeros(300)  # Return a zero vector if no words are present
    else:
        return model_own.wv.get_mean_vector(words)  # Compute the mean vector using Word2Vec model

X_train_w2v_own = []
X_test_w2v_own = []

# Extract Word2Vec features for training data
for e in tqdm(X_train_nltk_preprocessed):
    X_train_w2v_own.append(get_doc_embedding(e))

# Extract Word2Vec features for testing data
for e in tqdm(X_test):
    X_test_w2v_own.append(get_doc_embedding(e))
    
X_train_w2v_own = np.array(X_train_w2v_own)
X_test_w2v_own = np.array(X_test_w2v_own)

X_train_w2v_own.shape, X_test_w2v_own.shape

100%|███████████████████████████████████| 50000/50000 [00:42<00:00, 1168.12it/s]


((200000, 300), (50000, 300))

In [32]:
def get_doc_embedding(doc):
    doc_str = ' '.join(doc)  # Join tokens within the document list to form a single string
    words = doc_str.lower().split()  # Split the string into words
    if not words:
        return np.zeros(300)  # Return a zero vector if no words are present
    else:
        return wv.get_mean_vector(words)  # Compute the mean vector using Word2Vec model

X_train_w2v = []
X_test_w2v = []

# Extract Word2Vec features for training data
for e in tqdm(X_train_nltk_preprocessed):
    X_train_w2v.append(get_doc_embedding(e))

# Extract Word2Vec features for testing data
for e in tqdm(X_test):
    X_test_w2v.append(get_doc_embedding(e))
    
X_train_w2v = np.array(X_train_w2v)
X_test_w2v = np.array(X_test_w2v)

X_train_w2v.shape, X_test_w2v.shape

100%|███████████████████████████████████| 50000/50000 [00:46<00:00, 1085.76it/s]


((200000, 300), (50000, 300))

In [33]:
# Convert labels to NumPy arrays
y_train_np = np.array(y_train)
y_test_np = np.array(y_test)

# Convert labels to integers ranging from 0 to 2
y_train_encoded = [label - 1 for label in y_train_np]  # Assuming labels start from 1
y_test_encoded = [label - 1 for label in y_test_np]  # Assuming labels start from 1

# Define the MLP Model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return self.softmax(x)

# Training Loop
def train(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss}")

# Evaluation
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy: {correct/total}")

## Extract the Word2Vec Pre-trained Features and train a Multi Layer Perceptron

In [34]:
# Convert data to PyTorch tensors
train_dataset_pre = TensorDataset(torch.tensor(X_train_w2v, dtype=torch.float32), torch.tensor(y_train_encoded, dtype=torch.long))
test_dataset_pre = TensorDataset(torch.tensor(X_test_w2v, dtype=torch.float32), torch.tensor(y_test_encoded, dtype=torch.long))

# Create DataLoader
train_loader_pre = DataLoader(train_dataset_pre, batch_size=32, shuffle=True)
test_loader_pre = DataLoader(test_dataset_pre, batch_size=32)

# Initialize the model
input_size = X_train_w2v.shape[1]  # Size of input features
hidden_size1 = 50
hidden_size2 = 10
output_size = 3  # Assuming 3 classes for sentiment analysis
model_pre = MLP(input_size, hidden_size1, hidden_size2, output_size)

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_pre.parameters(), lr=0.001)

# Train the model
num_epochs = 50
train(model_pre, train_loader_pre, criterion, optimizer, num_epochs)

# Evaluate the model
evaluate(model_pre, test_loader_pre)

Epoch 1/50, Loss: 6507.37576764822
Epoch 2/50, Loss: 6392.508574426174
Epoch 3/50, Loss: 6363.861379146576
Epoch 4/50, Loss: 6349.22602045536
Epoch 5/50, Loss: 6339.5665445923805
Epoch 6/50, Loss: 6336.605700433254
Epoch 7/50, Loss: 6328.115510582924
Epoch 8/50, Loss: 6318.499319434166
Epoch 9/50, Loss: 6308.493044793606
Epoch 10/50, Loss: 6300.7000160217285
Epoch 11/50, Loss: 6296.519177913666
Epoch 12/50, Loss: 6291.990942358971
Epoch 13/50, Loss: 6287.559504747391
Epoch 14/50, Loss: 6283.4482543468475
Epoch 15/50, Loss: 6280.895238697529
Epoch 16/50, Loss: 6277.094581604004
Epoch 17/50, Loss: 6271.9096955657005
Epoch 18/50, Loss: 6272.777599990368
Epoch 19/50, Loss: 6267.2739517092705
Epoch 20/50, Loss: 6268.0393434762955
Epoch 21/50, Loss: 6265.1053104400635
Epoch 22/50, Loss: 6263.362920105457
Epoch 23/50, Loss: 6261.958479940891
Epoch 24/50, Loss: 6258.91188788414
Epoch 25/50, Loss: 6257.381070971489
Epoch 26/50, Loss: 6257.880622267723
Epoch 27/50, Loss: 6255.746350705624
Epoch 

## Extract the Word2Vec Custom Features and train a Multi Layer Perceptron

In [35]:
# Convert data to PyTorch tensors
train_dataset_own = TensorDataset(torch.tensor(X_train_w2v_own, dtype=torch.float32), torch.tensor(y_train_encoded, dtype=torch.long))
test_dataset_own = TensorDataset(torch.tensor(X_test_w2v_own, dtype=torch.float32), torch.tensor(y_test_encoded, dtype=torch.long))

# Create DataLoader
train_loader_own = DataLoader(train_dataset_own, batch_size=32, shuffle=True)
test_loader_own = DataLoader(test_dataset_own, batch_size=32)

# Initialize the model
input_size = X_train_w2v_own.shape[1]  # Size of input features
hidden_size1 = 50
hidden_size2 = 10
output_size = 3  # Assuming 3 classes for sentiment analysis
model_own = MLP(input_size, hidden_size1, hidden_size2, output_size)

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_own.parameters(), lr=0.001)

# Train the model
num_epochs = 50
train(model_own, train_loader_own, criterion, optimizer, num_epochs)

# Evaluate the model
evaluate(model_own, test_loader_own)

Epoch 1/50, Loss: 6506.918885469437
Epoch 2/50, Loss: 6399.554023325443
Epoch 3/50, Loss: 6377.080897450447
Epoch 4/50, Loss: 6363.953673005104
Epoch 5/50, Loss: 6355.022880017757
Epoch 6/50, Loss: 6348.933548569679
Epoch 7/50, Loss: 6341.789035379887
Epoch 8/50, Loss: 6332.161166250706
Epoch 9/50, Loss: 6324.765000879765
Epoch 10/50, Loss: 6319.54182434082
Epoch 11/50, Loss: 6313.848741412163
Epoch 12/50, Loss: 6310.189614236355
Epoch 13/50, Loss: 6307.424668550491
Epoch 14/50, Loss: 6303.995420217514
Epoch 15/50, Loss: 6300.692300200462
Epoch 16/50, Loss: 6297.822930276394
Epoch 17/50, Loss: 6295.129075586796
Epoch 18/50, Loss: 6290.334086894989
Epoch 19/50, Loss: 6288.532537519932
Epoch 20/50, Loss: 6285.952550649643
Epoch 21/50, Loss: 6283.689514100552
Epoch 22/50, Loss: 6282.665510833263
Epoch 23/50, Loss: 6279.689943790436
Epoch 24/50, Loss: 6277.734576165676
Epoch 25/50, Loss: 6277.1504472494125
Epoch 26/50, Loss: 6274.926438689232
Epoch 27/50, Loss: 6273.028074383736
Epoch 28/5

## Using the First 10 Word2Vec vectors on Ternary Classification for the Word2Vec Custom Model

In [43]:
# Concatenate the first 10 Word2Vec vectors for each review
X_train_concat_own = []
X_test_concat_own = []

for review in X_train_w2v_own:
    review_reshaped = review.reshape(1, -1)  # Reshape to ensure it's a 2D array
    concatenated_vector = np.concatenate(review_reshaped[:10], axis=0)
    X_train_concat_own.append(concatenated_vector)

for review in X_test_w2v_own:
    review_reshaped = review.reshape(1, -1)  # Reshape to ensure it's a 2D array
    concatenated_vector = np.concatenate(review_reshaped[:10], axis=0)
    X_test_concat_own.append(concatenated_vector)

# Convert the concatenated features into PyTorch tensors
X_train_concat_tensor_own = torch.tensor(X_train_concat_own, dtype=torch.float32)
X_test_concat_tensor_own = torch.tensor(X_test_concat_own, dtype=torch.float32)

# Convert labels to ternary format
y_train_tensor_own = torch.tensor(y_train_encoded, dtype=torch.long)
y_test_tensor_own = torch.tensor(y_test_encoded, dtype=torch.long)

# Create DataLoader
train_dataset_concat_own = TensorDataset(X_train_concat_tensor_own, y_train_tensor_own)
test_dataset_concat_own = TensorDataset(X_test_concat_tensor_own, y_test_tensor_own)

train_loader_concat_own = DataLoader(train_dataset_concat_own, batch_size=32, shuffle=True)
test_loader_concat_own = DataLoader(test_dataset_concat_own, batch_size=32)

# Initialize the model
input_size = X_train_concat_tensor_own.shape[1]  # Size of input features
hidden_size1 = 50
hidden_size2 = 10
output_size = 3  # Assuming 3 classes for sentiment analysis
model_concat_own = MLP(input_size, hidden_size1, hidden_size2, output_size)

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_concat_own.parameters(), lr=0.001)

# Train the model
num_epochs = 50
train(model_concat_own, train_loader_concat_own, criterion, optimizer, num_epochs)

# Evaluate the model
evaluate(model_concat_own, test_loader_concat_own)

Epoch 1/50, Loss: 6493.512466669083
Epoch 2/50, Loss: 6403.701088666916
Epoch 3/50, Loss: 6375.310911118984
Epoch 4/50, Loss: 6358.125028192997
Epoch 5/50, Loss: 6349.976001679897
Epoch 6/50, Loss: 6341.9753767848015
Epoch 7/50, Loss: 6334.247457385063
Epoch 8/50, Loss: 6327.67810690403
Epoch 9/50, Loss: 6322.453595340252
Epoch 10/50, Loss: 6319.002834439278
Epoch 11/50, Loss: 6313.617429792881
Epoch 12/50, Loss: 6308.798898279667
Epoch 13/50, Loss: 6305.797322332859
Epoch 14/50, Loss: 6304.67782998085
Epoch 15/50, Loss: 6301.674203693867
Epoch 16/50, Loss: 6299.178352296352
Epoch 17/50, Loss: 6297.800688326359
Epoch 18/50, Loss: 6294.289004981518
Epoch 19/50, Loss: 6291.694235086441
Epoch 20/50, Loss: 6287.282687723637
Epoch 21/50, Loss: 6283.074702978134
Epoch 22/50, Loss: 6279.768384814262
Epoch 23/50, Loss: 6277.975959420204
Epoch 24/50, Loss: 6275.396085143089
Epoch 25/50, Loss: 6272.225798428059
Epoch 26/50, Loss: 6270.133328974247
Epoch 27/50, Loss: 6268.396264076233
Epoch 28/50

## Using the First 10 Word2Vec vectors on Ternary Classification for the Word2Vec Pre-Trained Model

In [44]:
# Concatenate the first 10 Word2Vec vectors for each review
X_train_concat = []
X_test_concat = []

for review in X_train_w2v:
    review_reshaped = review.reshape(1, -1)  # Reshape to ensure it's a 2D array
    concatenated_vector = np.concatenate(review_reshaped[:10], axis=0)
    X_train_concat.append(concatenated_vector)

for review in X_test_w2v:
    review_reshaped = review.reshape(1, -1)  # Reshape to ensure it's a 2D array
    concatenated_vector = np.concatenate(review_reshaped[:10], axis=0)
    X_test_concat.append(concatenated_vector)

# Convert the concatenated features into PyTorch tensors
X_train_concat_tensor = torch.tensor(X_train_concat, dtype=torch.float32)
X_test_concat_tensor = torch.tensor(X_test_concat, dtype=torch.float32)

# Convert labels to ternary format
y_train_tensor = torch.tensor(y_train_encoded, dtype=torch.long)
y_test_tensor = torch.tensor(y_test_encoded, dtype=torch.long)

# Create DataLoader
train_dataset_concat = TensorDataset(X_train_concat_tensor, y_train_tensor)
test_dataset_concat = TensorDataset(X_test_concat_tensor, y_test_tensor)

train_loader_concat = DataLoader(train_dataset_concat, batch_size=32, shuffle=True)
test_loader_concat = DataLoader(test_dataset_concat, batch_size=32)

# Initialize the model
input_size = X_train_concat_tensor.shape[1]  # Size of input features
hidden_size1 = 50
hidden_size2 = 10
output_size = 3  # Assuming 3 classes for sentiment analysis
model_concat = MLP(input_size, hidden_size1, hidden_size2, output_size)

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_concat.parameters(), lr=0.001)

# Train the model
num_epochs = 50
train(model_concat, train_loader_concat, criterion, optimizer, num_epochs)

# Evaluate the model
evaluate(model_concat, test_loader_concat)

Epoch 1/50, Loss: 6518.244131207466
Epoch 2/50, Loss: 6420.93543356657
Epoch 3/50, Loss: 6396.908332765102
Epoch 4/50, Loss: 6374.263934910297
Epoch 5/50, Loss: 6359.016069591045
Epoch 6/50, Loss: 6348.8051480054855
Epoch 7/50, Loss: 6340.967021346092
Epoch 8/50, Loss: 6335.828918457031
Epoch 9/50, Loss: 6331.106681883335
Epoch 10/50, Loss: 6325.491824150085
Epoch 11/50, Loss: 6323.024015903473
Epoch 12/50, Loss: 6318.552897572517
Epoch 13/50, Loss: 6316.903543651104
Epoch 14/50, Loss: 6311.946495294571
Epoch 15/50, Loss: 6306.8835836052895
Epoch 16/50, Loss: 6303.869810521603
Epoch 17/50, Loss: 6300.620277702808
Epoch 18/50, Loss: 6298.602122366428
Epoch 19/50, Loss: 6296.123710632324
Epoch 20/50, Loss: 6293.452532947063
Epoch 21/50, Loss: 6292.05080384016
Epoch 22/50, Loss: 6289.058473885059
Epoch 23/50, Loss: 6287.096545815468
Epoch 24/50, Loss: 6284.060394287109
Epoch 25/50, Loss: 6282.42000490427
Epoch 26/50, Loss: 6279.200699985027
Epoch 27/50, Loss: 6277.83346247673
Epoch 28/50,

## Using the First 10 Word2Vec vectors on Binary Classification for the Word2Vec Pre-Trained Model

In [50]:
# Concatenate the first 10 Word2Vec vectors for each review
X_train_concat = []
X_test_concat = []

for review in X_train_w2v_binary:
    review_reshaped = review.reshape(1, -1)  # Reshape to ensure it's a 2D array
    concatenated_vector = np.concatenate(review_reshaped[:10], axis=0)
    X_train_concat.append(concatenated_vector)

for review in X_test_w2v_binary:
    review_reshaped = review.reshape(1, -1)  # Reshape to ensure it's a 2D array
    concatenated_vector = np.concatenate(review_reshaped[:10], axis=0)
    X_test_concat.append(concatenated_vector)

# Convert the concatenated features into PyTorch tensors
X_train_concat_tensor = torch.tensor(X_train_concat, dtype=torch.float32)
X_test_concat_tensor = torch.tensor(X_test_concat, dtype=torch.float32)

# Convert labels to ternary format
y_train_tensor = torch.tensor(y_train_binary, dtype=torch.long)
y_test_tensor = torch.tensor(y_test_binary, dtype=torch.long)

# Create DataLoader
train_dataset_concat = TensorDataset(X_train_concat_tensor, y_train_tensor)
test_dataset_concat = TensorDataset(X_test_concat_tensor, y_test_tensor)

train_loader_concat = DataLoader(train_dataset_concat, batch_size=32, shuffle=True)
test_loader_concat = DataLoader(test_dataset_concat, batch_size=32)

# Initialize the model
input_size = X_train_concat_tensor.shape[1]  # Size of input features
hidden_size1 = 50
hidden_size2 = 10
output_size = 2  # Assuming 2 classes for sentiment analysis
model_concat = MLP(input_size, hidden_size1, hidden_size2, output_size)

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_concat.parameters(), lr=0.001)

# Train the model
num_epochs = 50
train(model_concat, train_loader_concat, criterion, optimizer, num_epochs)

# Evaluate the model
evaluate(model_concat, test_loader_concat)

Epoch 1/50, Loss: 3342.113238632679
Epoch 2/50, Loss: 3262.320921331644
Epoch 3/50, Loss: 3237.025598526001
Epoch 4/50, Loss: 3223.482433617115
Epoch 5/50, Loss: 3216.778574824333
Epoch 6/50, Loss: 3212.782517582178
Epoch 7/50, Loss: 3209.696006208658
Epoch 8/50, Loss: 3205.975496381521
Epoch 9/50, Loss: 3203.7256236970425
Epoch 10/50, Loss: 3201.2478472590446
Epoch 11/50, Loss: 3198.1952142715454
Epoch 12/50, Loss: 3193.772346884012
Epoch 13/50, Loss: 3193.0757144391537
Epoch 14/50, Loss: 3191.648870229721
Epoch 15/50, Loss: 3187.7114550173283
Epoch 16/50, Loss: 3187.3979715406895
Epoch 17/50, Loss: 3182.9758153259754
Epoch 18/50, Loss: 3178.705210506916
Epoch 19/50, Loss: 3173.8695701658726
Epoch 20/50, Loss: 3170.260470122099
Epoch 21/50, Loss: 3167.7650876045227
Epoch 22/50, Loss: 3167.2493092119694
Epoch 23/50, Loss: 3166.244423389435
Epoch 24/50, Loss: 3165.200751185417
Epoch 25/50, Loss: 3164.42650526762
Epoch 26/50, Loss: 3163.0863120555878
Epoch 27/50, Loss: 3161.6579765081406

## Using the First 10 Word2Vec vectors on Binary Classification for the Word2Vec Custom Model

In [52]:
# Concatenate the first 10 Word2Vec vectors for each review
X_train_concat = []
X_test_concat = []

for review in X_train_w2v_own_binary:
    review_reshaped = review.reshape(1, -1)  # Reshape to ensure it's a 2D array
    concatenated_vector = np.concatenate(review_reshaped[:10], axis=0)
    X_train_concat.append(concatenated_vector)

for review in X_test_w2v_own_binary:
    review_reshaped = review.reshape(1, -1)  # Reshape to ensure it's a 2D array
    concatenated_vector = np.concatenate(review_reshaped[:10], axis=0)
    X_test_concat.append(concatenated_vector)

# Convert the concatenated features into PyTorch tensors
X_train_concat_tensor = torch.tensor(X_train_concat, dtype=torch.float32)
X_test_concat_tensor = torch.tensor(X_test_concat, dtype=torch.float32)

# Convert labels to ternary format
y_train_tensor = torch.tensor(y_train_binary, dtype=torch.long)
y_test_tensor = torch.tensor(y_test_binary, dtype=torch.long)

# Create DataLoader
train_dataset_concat = TensorDataset(X_train_concat_tensor, y_train_tensor)
test_dataset_concat = TensorDataset(X_test_concat_tensor, y_test_tensor)

train_loader_concat = DataLoader(train_dataset_concat, batch_size=32, shuffle=True)
test_loader_concat = DataLoader(test_dataset_concat, batch_size=32)

# Initialize the model
input_size = X_train_concat_tensor.shape[1]  # Size of input features
hidden_size1 = 50
hidden_size2 = 10
output_size = 2  # Assuming 2 classes for sentiment analysis
model_concat = MLP(input_size, hidden_size1, hidden_size2, output_size)

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_concat.parameters(), lr=0.001)

# Train the model
num_epochs = 50
train(model_concat, train_loader_concat, criterion, optimizer, num_epochs)

# Evaluate the model
evaluate(model_concat, test_loader_concat)

Epoch 1/50, Loss: 3332.658839702606
Epoch 2/50, Loss: 3261.719752550125
Epoch 3/50, Loss: 3233.302696377039
Epoch 4/50, Loss: 3221.9010125100613
Epoch 5/50, Loss: 3212.4864708781242
Epoch 6/50, Loss: 3204.434473723173
Epoch 7/50, Loss: 3199.1797445118427
Epoch 8/50, Loss: 3192.838485687971
Epoch 9/50, Loss: 3188.5063311755657
Epoch 10/50, Loss: 3183.40127363801
Epoch 11/50, Loss: 3181.311291396618
Epoch 12/50, Loss: 3177.702940106392
Epoch 13/50, Loss: 3174.968383014202
Epoch 14/50, Loss: 3173.754948168993
Epoch 15/50, Loss: 3170.6849725842476
Epoch 16/50, Loss: 3169.7179051041603
Epoch 17/50, Loss: 3167.712255001068
Epoch 18/50, Loss: 3165.444971472025
Epoch 19/50, Loss: 3165.503632068634
Epoch 20/50, Loss: 3164.026943206787
Epoch 21/50, Loss: 3164.1536640822887
Epoch 22/50, Loss: 3160.7471270263195
Epoch 23/50, Loss: 3161.133596032858
Epoch 24/50, Loss: 3159.1496563851833
Epoch 25/50, Loss: 3156.0669295191765
Epoch 26/50, Loss: 3155.8342953920364
Epoch 27/50, Loss: 3153.715820401907
